### **Intro**

See [here](http://web.mit.edu/julia_v0.6.2/julia/share/doc/julia/html/en/manual/performance-tips.html) for more info about getting performance out of Julia. 

In [37]:
using BenchmarkTools

function innerDot(a::Vector{Float64}, b::Vector{Float64})
    l = min(length(a), length(b)) ::Int64
    s = 0.0
    @simd for II in 1:l
        @inbounds s += a[II]*b[II]
    end
    return s
end

function innerdotSlow(a, b)
    l = min(length(a), length(b))
    s = 0
    for II in 1:l
        s += a[II]*b[II]
    end
    return s
end
# For compilation
innerDot([1.1, 2,2], [2.2, 2.2])
innerdotSlow([1.1, 2,2], [2.2, 2.2])

6.82

In [52]:
GC.gc() 
function timeIt()
    N = 2^20
    a, b = rand(N), rand(N)
    println("Random vector Generated ")
    GC.gc() 
    @time for II in 1:100
         innerDot(a, b)
    end
    
end
timeIt()

Random vector Generated 
  0.000000 seconds


In [63]:
function inner(x, y)
    s = zero(eltype(x))
    for i=1:length(x)
        @inbounds s += x[i]*y[i]
    end
    s
end

function innersimd(x, y)
    s = zero(eltype(x))
    @simd for i=1:length(x)
        @inbounds s += x[i]*y[i]
    end
    s
end

function timeit(n, reps)
    x = rand(Float32,n)
    y = rand(Float32,n)
    s = zero(Float64)
    @time for j in 1:reps
        s+=inner(x,y)
    end
    # println("GFlop/sec        = ",2.0*n*reps/time*1E-9)
    # println("time = ", time)
    @time for j in 1:reps
        s+=innersimd(x,y)
    end
    # println("GFlop/sec (SIMD) = ",2.0*n*reps/time*1E-9)
    # println("time (SIMD) = ", time)
end

timeit(2^23,2^2)

  0.000000 seconds
  0.000000 seconds
